In [1]:
import fast_stats
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

In [2]:
SIZE = (10, 512, 512)

In [ ]:
y_true, y_pred = np.random.randint(0, 4, SIZE), np.random.randint(0, 4, SIZE)

In [ ]:
fast_stats._confusion_matrix(y_true, y_pred, [0, 1, 2, 3, 4, 5])

In [ ]:
%%timeit
_ = fast_stats._confusion_matrix(y_true, y_pred, [0, 1, 2, 3, 4, 5])

In [16]:
mat = np.random.randint(0, 4, SIZE).astype(np.int64)

In [17]:
%%timeit
_ = fast_stats._unique(mat)

28.4 ms ± 69.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
_ = np.unique(mat)

34.5 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Compared to scikit-learn

In [ ]:
pred = np.random.randint(0, 2, SIZE).astype(bool).flatten()
actual = np.random.randint(0, 2, SIZE).astype(bool).flatten()

In [ ]:
%%timeit
_ = precision_score(actual, pred)

In [ ]:
%%timeit
_ = recall_score(actual, pred)

In [ ]:
%%timeit
_ = f1_score(actual, pred)

In [ ]:
# don't actually need to flatten them for fast-stats
pred = np.random.randint(0, 2, SIZE).astype(bool)
actual = np.random.randint(0, 2, SIZE).astype(bool)

In [ ]:
%%timeit
_ = fast_stats.binary_precision(actual, pred)

In [ ]:
%%timeit
_ = fast_stats.binary_recall(actual, pred)

In [ ]:
%%timeit
_ = fast_stats.binary_f1_score(actual, pred)

In [ ]:
assert np.allclose(
    fast_stats.binary_precision(actual.flatten(), pred.flatten()),
    precision_score(actual.flatten(), pred.flatten())
)
assert np.allclose(
    fast_stats.binary_recall(actual.flatten(), pred.flatten()),
    recall_score(actual.flatten(), pred.flatten())
)
assert np.allclose(
    fast_stats.binary_f1_score(actual.flatten(), pred.flatten()),
    f1_score(actual.flatten(), pred.flatten())
)

### Compared to numpy

In [ ]:
pred, actual = pred.astype(np.uint8), actual.astype(np.uint8)

In [ ]:
%%timeit
_ = fast_stats.binary_precision(pred, actual)

In [ ]:
%%timeit
_ = np.logical_and(actual, pred).sum() / pred.sum()